Nettoyage des données des dépenses publiques sportives pour les pays européens

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys

# Définir le chemin racine du projet
project_root = os.path.abspath("/Users/sacharatsavong/Desktop/Python/projet_python_roland_sacha")
sys.path.append(project_root)

# Changer le directory courant
os.chdir(project_root)

In [3]:
dep = pd.read_csv("data/raw/gov_10a_exp$defaultview_linear_2_0.csv")

In [ ]:
# Conservation des variables d'intérêt

dep_small = dep[["Entité géopolitique (déclarante)", 'sector', 'Classification des fonctions des administrations publiques (CFAP 1999)', 
                "TIME_PERIOD", "OBS_VALUE"]]

# Filtrage pour ne garder que les dépenses liées aux loisirs, culture et culte

dep_small = dep_small[dep_small['Classification des fonctions des administrations publiques (CFAP 1999)'] == 'Loisirs, culture et culte']

# Agrégation des dépenses par pays et par année dans les différents secteurs

dep_small = dep_small.groupby(['Entité géopolitique (déclarante)', 'TIME_PERIOD'], as_index=False)['OBS_VALUE'].sum()

In [ ]:
# Dépenses moyennes sur les quatre années précédant les JO ciblés
dep_small['Dépenses'] = dep_small['OBS_VALUE']
dep_small['TIME_PERIOD'] = dep_small['TIME_PERIOD'].astype(int)

jo_years = [2016, 2021, 2025]
dep_small['Dépenses moyennes'] = pd.NA

for jo_year in jo_years:
    mask_pre_jo = dep_small['TIME_PERIOD'].between(jo_year - 4, jo_year - 1)
    depenses_moyennes = (
        dep_small.loc[mask_pre_jo]
        .groupby('Entité géopolitique (déclarante)')['Dépenses']
        .mean()
    )
    mask_jo_year = dep_small['TIME_PERIOD'] == jo_year
    dep_small.loc[mask_jo_year, 'Dépenses moyennes'] = dep_small.loc[mask_jo_year, 'Entité géopolitique (déclarante)'].map(depenses_moyennes)

dep_small[dep_small['TIME_PERIOD'].isin(jo_years)].head()


In [63]:
dep_small.head()

,Entité géopolitique (déclarante),TIME_PERIOD,OBS_VALUE
0,Allemagne,2015,2.0
1,Allemagne,2016,2.0
2,Allemagne,2017,2.0
3,Allemagne,2018,2.0
4,Allemagne,2019,2.0


In [64]:
dep_small.to_csv("data/raw/depenses_sports_UE.csv", index=False)